In [ ]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requ

In [ ]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [ ]:
import os
import openai
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

In [ ]:
pdfreader = PdfReader('/content/Byte - downloads.pdf')

In [ ]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [ ]:
len(raw_text)

6995

In [ ]:
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
document_search = FAISS.from_texts(texts, embeddings)

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [ ]:
chain = load_qa_chain(OpenAI(model_name="gpt-3.5-turbo-instruct",), chain_type="stuff")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [ ]:
query = "Create all possible questions from the given document"#Don't say ten questions(maybe no questions or just one question)
docs = document_search.similarity_search(query)#Look into max tokens(update it(gpt works such that the total sum of input and output is 4k))
question = chain.run(input_documents=docs, question=query)#batching(context might get missed)
question

'\n1. Is the device supported for downloading content?\n2. Is the device connected to the internet?\n3. Is the "Download" option available for the specific title?\n4. Where is the customer located?\n5. Are they located outside of the eligible region?\n6. Is the internet speed sufficient for downloading?\n7. Does the device have enough storage space for downloading?\n8. Did the viewer close the app while downloading?\n9. Is the "Download only on Wi-Fi for this device" option turned on?\n10. How can we assist the viewer?\n11. What should we do in this situation?\n12. Can you describe what you are experiencing?\n13. Are you getting any error messages? If so, what do they say?\n14. Is the content not playing at all?\n15. Is the content buffering frequently?\n16. Is the app crashing back to the home screen?\n17. Should we delete the downloaded content and try downloading it again?\n18. What basic troubleshooting steps can we perform?\n19. Should we fully close and restart the app?\n20. Shou

In [ ]:
question
# Split the string into a list using '\n' as the separator
questions_list = question.split('\n')

# Remove empty strings and create a list of questions
questions = [question.strip() for question in questions_list if question.strip()]

# Print the result
questions

['1. Is the device supported for downloading content?',
 '2. Is the device connected to the internet?',
 '3. Is the "Download" option available for the specific title?',
 '4. Where is the customer located?',
 '5. Are they located outside of the eligible region?',
 '6. Is the internet speed sufficient for downloading?',
 '7. Does the device have enough storage space for downloading?',
 '8. Did the viewer close the app while downloading?',
 '9. Is the "Download only on Wi-Fi for this device" option turned on?',
 '10. How can we assist the viewer?',
 '11. What should we do in this situation?',
 '12. Can you describe what you are experiencing?',
 '13. Are you getting any error messages? If so, what do they say?',
 '14. Is the content not playing at all?',
 '15. Is the content buffering frequently?',
 '16. Is the app crashing back to the home screen?',
 '17. Should we delete the downloaded content and try downloading it again?',
 '18. What basic troubleshooting steps can we perform?',
 '19.

In [ ]:
answers = []
for q in questions:
  query = q
  docs = document_search.similarity_search(query)
  answer = chain.run(input_documents=docs, question=query)
  answers.append(answer)

In [ ]:
answers

[' The author suggests organizing quiz questions by using the question/evidence/conclusion format from your notes.',
 ' The author recommends being flexible and finding a structure that works for combining or breaking up questions when creating quizzes. They suggest breaking up broad questions into smaller questions or combining small questions into larger ones, depending on the notes taken. Ultimately, the goal is to create practice quizzes that cover all the material contained in each corresponding chapter. ',
 ' The author suggests creating practice quizzes based on the notes taken for each chapter, breaking down broad questions into smaller ones and combining small questions into larger ones if necessary. They also recommend going through the quizzes and articulating the matching conclusion and supporting evidence, preferably out loud. ',
 ' On Monday and Tuesday, Julie spends two hours each day mastering the first two quizzes by pacing around her dorm room and lecturing answers to